In [77]:
import requests
import json
import PIL
import markdownify
import time
import re
import numpy as np

In [74]:
sharecodes = ["2QBZH2", "IGPIC3", "IBWY9V", "XQM6VW", "SSFQFB", "3XE93H", "863UIR", "BPAIBK", "9RZWZF", "UAFVRY", "JYG92Y", "YS27SL", "CJACUP", "QVIYBF"]
datas = []
for code in sharecodes:
    shareCodeResponce = json.loads(requests.get(f"https://api.compcon.app/share?code={code}", headers={"x-api-key": "fcFvjjrnQy2hypelJQi4X9dRI55r5KuI4bC07Maf"}).text)
    AWS_url = shareCodeResponce["presigned"]
    data = json.loads(requests.get(AWS_url).text)
    datas.append(data)
    time.sleep(2)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [68]:
img_lookup = {
    'mf_black_witch_alt_orchidea': 'orchis'
}
corpo_lookup = {'mf_chomolungma': 'GMS',
 'mf_standard_pattern_i_everest': 'GMS',
 'mf_zheng': 'IPS-N',
 'mf_gorgon': 'HORUS',
 'mf_hydra': 'HORUS',
 'mf_blackbeard': 'IPS-N',
 'mf_balor': 'HORUS',
 'mf_black_witch_alt_orchidea': 'SSC'}

In [69]:
def get_vue_bits(data):
    BASE='public'
    callsign = data["callsign"]
    print(callsign)
    alias = data["name"]
    code = data["id"]
    
    
    history = data["history"]
    with open(f"{BASE}/pilots/{callsign}.md", "w+") as f:
        f.write(markdownify.markdownify(history))
        
    if data["cloud_portrait"] != '':
        img = data["cloud_portrait"]
    elif data["portrait"] != '':
        img = f"https://d2c79xe1p61csc.cloudfront.net/pilots/{data['portrait']}"
    else:
        img = f'https://compcon.app/static/img/pilot/nodata.png'
    img_data = PIL.Image.open(requests.get(img, stream=True).raw)
    img_data.save(f"{BASE}/pilots/{callsign}.png")    
    
    active_id = data['state']['active_mech_id']
    mechs = data["mechs"]
    for i, mech in enumerate(mechs):
        if mech["id"] == active_id:   
            mech_n = i
            break
    else:
        mech_n = -1
    
    mech = mechs[mech_n]
    name = mech['name']
    frame = mech['frame']
    corpo = corpo_lookup[frame]
    
    if mech["cloud_portrait"] != '':
        img = mech["cloud_portrait"]
    elif mech["portrait"] != '':
        print('Warning portraight mech')
        print(mech["portrait"])
        pass
    else:
        if frame in img_lookup:
            img = f'https://d2c79xe1p61csc.cloudfront.net/frames/{img_lookup[frame]}.png'
        else:
            img = f'https://d2c79xe1p61csc.cloudfront.net/frames/{frame}.png'
    try:
        img_data = PIL.Image.open(requests.get(img, stream=True).raw)
    except:
        print(f'MEch Image Fail')
        img = f'https://d2c79xe1p61csc.cloudfront.net/frames/nodata.png'
        img_data = PIL.Image.open(requests.get(img, stream=True).raw)
    img_data.save(f"{BASE}/mechs/{name}.png")
    
    vue_text = f'{{\n\t"callsign": "{callsign}",\n\t"alias": "{alias}",\n\t"code": "{code}",\n\t"corpo": "{corpo}",\n\t"frame": "{frame.strip("mf_").replace("_", " ")}",\n\t"mech": "{name}"\n}},'
    
    return vue_text

In [78]:
vue_text = ""

LLs = []
for data in datas:
    LLs.append(data['level'])
order = np.argsort(LLs)
for i in order[::-1]:
    data = datas[i]
    vue_text += get_vue_bits(data) + '\n'

Monkey
HACKERMANS.omif
Charon
Birdy
MIRAGE
Lockjaw
Badger
HB
Foundryside
Khepri
Ember
MEch Image Fail
Castlefall
MEch Image Fail
Count


In [79]:
with open('src/App.vue', 'r') as f:
    app = f.read()
    
pilots_re = re.compile(r'"pilots": \[\n([\s\S]*)]')

span = pilots_re.search(app).span(1)
out = app[:span[0]] + vue_text + app[span[1]:]

In [72]:
with open('src/App.vue', 'w+') as f:
    f.write(out)

In [59]:
frames = {}
for data in datas:
    active_id = data['state']['active_mech_id']
    mechs = data["mechs"]
    for i, mech in enumerate(mechs):
        if mech["id"] == active_id:   
            mech_n = i
            break
    else:
        mech_n = -1
    
    mech = mechs[mech_n]
    frame = mech['frame']
    if frame not in frames.keys():
        frames[frame] = input(f'{frame} > ')

mf_chomolungma > GMS
mf_standard_pattern_i_everest > GMS
mf_zheng > IPS-N
mf_gorgon > HORUS
mf_hydra > HORUS
mf_blackbeard > IPS-N
mf_balor > HORUS
mf_black_witch_alt_orchidea > SSC


In [60]:
frames

{'mf_chomolungma': 'GMS',
 'mf_standard_pattern_i_everest': 'GMS',
 'mf_zheng': 'IPS-N',
 'mf_gorgon': 'HORUS',
 'mf_hydra': 'HORUS',
 'mf_blackbeard': 'IPS-N',
 'mf_balor': 'HORUS',
 'mf_black_witch_alt_orchidea': 'SSC'}

In [75]:
data.keys()

dict_keys(['id', 'level', 'callsign', 'name', 'player_name', 'status', 'dead', 'text_appearance', 'notes', 'history', 'quirks', 'current_hp', 'background', 'resistances', 'mechs', 'cc_ver', 'special_equipment', 'combat_history', 'state', 'lastModified', 'isDeleted', 'expireTime', 'deleteTime', 'lastUpdate_cloud', 'lastSync', 'shareCode', 'shareCodeExpiry', 'isRemoteResource', 'skills', 'talents', 'mechSkills', 'counter_data', 'custom_counters', 'core_bonuses', 'licenses', 'reserves', 'orgs', 'bondId', 'xp', 'stress', 'isBroken', 'burdens', 'bondPowers', 'powerSelections', 'maxStress', 'bondAnswers', 'minorIdeal', 'clocks', 'group', 'sort_index', 'portrait', 'cloud_portrait', 'loadout', 'brews'])